**Imposting the Required Library**

In [194]:
from numpy import *
from pandas import *

**Defining the Functions**

In [195]:
def moment(m1,m2,r1):
    return ((m1*m2)/(m1+m2))* r1**2

def moment2(mo,ms,mc,rco,rcs):
    M = mo+mc+ms
    I = mo*rco**2 + ms*rcs**2 - (((mo*rco - ms*rcs)**2)/M)
    return I

def B(I):
    c = 3e8
    h = 6.62e-34
    B = ((h)/(8*pi**2*I*c))/100
    return B

def D(I,r1,r2,k):
    h = 6.63e-34
    r = (r1 + r2)/2
    c = 3e8
    d = (h**3)/(32 * pi**4 * I**2 * r**2 * k * c)
    return d
def omega(B,D):
    o = sqrt((4 * B**3)/(D))
    return o
def Energy(n,j,B,D):
    E_vib = (n + 1/2)*omega(B,D)
    E_rot = B*(j*(j+1)) - D*(j**2*(j+1)**2)
    return E_vib + E_rot

**HF Molecule**

In [196]:
Energy_HF=[]
J = arange(0,10,1)
N = arange(0,3,1)
m1 = 1.672163981e-27
m2 = 3.154694272e-26
r1 = 9.27e-11
k = 96600
j = 0
b = B(moment(m1,m2,r1))
while j < len(J):
    Energy_HF.append(b*j*(j+1))
    j+=1

df = DataFrame({
    'J': J,
    'Energy_HF': Energy_HF
})

**Pure Rotational**

In [197]:
Transition_HF = []
Energy_difference_HF = []

for j in range(len(J)-1):
    Transition_HF.append(f'{j+1} --> {j}')
    Energy_difference_HF.append(Energy_HF[j+1]-Energy_HF[j])
    
Diff_HF = DataFrame({
    'Transition level' : Transition_HF,
    'Energy Differnece' : Energy_difference_HF
})

In [198]:
print(D(moment(m1,m2,r1),r1,r1,k))
print(B(moment(m1,m2,r1)))
print(omega(B(moment(m1,m2,r1)),D(moment(m1,m2,r1),r1,r1,k)))

0.0020161272805543987
20.48044279794816
4128.38351966667


In [199]:
print(df)
df.to_csv("Rotational HF.csv", index=False)
print(Diff_HF)
Diff_HF.to_csv("Transition_HF.csv", index=False)

   J    Energy_HF
0  0     0.000000
1  1    40.960886
2  2   122.882657
3  3   245.765314
4  4   409.608856
5  5   614.413284
6  6   860.178598
7  7  1146.904797
8  8  1474.591881
9  9  1843.239852
  Transition level  Energy Differnece
0          1 --> 0          40.960886
1          2 --> 1          81.921771
2          3 --> 2         122.882657
3          4 --> 3         163.843542
4          5 --> 4         204.804428
5          6 --> 5         245.765314
6          7 --> 6         286.726199
7          8 --> 7         327.687085
8          9 --> 8         368.647970


**Rotation and Vibrational**

In [200]:
Energy_rot_HF = []
b = B(moment(m1,m2,r1))
d = D(moment(m1,m2,r1),r1,r1,k)
for n in N:
    for j in J:
        Energy_rot_HF.append(Energy(n,j,b,d))


In [201]:
df = DataFrame({
    'N': [n for n in N for _ in J],
    'J': [j for _ in N for j in J],
    'Energy_rot_HF (1/cm)': Energy_rot_HF
})

In [202]:
Transition_vib_rot_HF = []
Transition_combination_HF = []
for k in range(len(Energy_rot_HF)):
    n1 = df.at[k, 'N']
    j1 = df.at[k,'J']
    for l in range(k,len(Energy_rot_HF)):
        n2 = df.at[l,'N']
        j2 = df.at[l,'J']
        if abs(n1 - n2) == 1 and abs(j2 - j1) == 1:
            tr = df.at[l , 'Energy_rot_HF (1/cm)'] - df.at[k , 'Energy_rot_HF (1/cm)']
            Transition_vib_rot_HF.append(tr)
            Transition_combination_HF.append(f'{n1} --> {n2} and {j1} --> {j2}')
            
Transition_comb_HF = DataFrame({
    'Transition' : Transition_combination_HF,
    'Difference in Energy level (1/cm)' : Transition_vib_rot_HF
})

In [203]:
print(df)
df.to_csv("Vibration_HF.csv", index=False)
print(Transition_comb_HF)
Transition_comb_HF.to_csv("Transition_vibration_HF.csv", index=False)

    N  J  Energy_rot_HF (1/cm)
0   0  0           2064.191760
1   0  1           2105.144581
2   0  2           2187.001836
3   0  3           2309.666751
4   0  4           2472.994165
5   0  5           2676.790529
6   0  6           2920.813909
7   0  7           3204.773981
8   0  8           3528.332037
9   0  9           3891.100981
10  1  0           6192.575280
11  1  1           6233.528101
12  1  2           6315.385356
13  1  3           6438.050271
14  1  4           6601.377685
15  1  5           6805.174049
16  1  6           7049.197428
17  1  7           7333.157501
18  1  8           7656.715557
19  1  9           8019.484500
20  2  0          10320.958799
21  2  1          10361.911620
22  2  2          10443.768875
23  2  3          10566.433790
24  2  4          10729.761204
25  2  5          10933.557569
26  2  6          11177.580948
27  2  7          11461.541021
28  2  8          11785.099077
29  2  9          12147.868020
             Transition  Difference in 

**SOC Molecule**

In [204]:
Energy_SOC = []
mo = 2.656698266e-26
mc = 1.994474834e-26
ms = 5.323691881e-26
roc = 0.116e-9
rsc = 0.155e-9
k = 20000
j = 0
b = B(moment2(mo,ms,mc,roc,rsc))
while j < len(J):
    Energy_SOC.append(b*j*(j+1))
    j+=1
    
df_SOC = DataFrame({
    'J': J,
    'Energy_HF': Energy_SOC
})

**Pure Rotational**

In [205]:
Transition_SOC = []
Energy_difference_SOC = []

for j in range(len(J)-1):
    Transition_SOC.append(f'{j+1} --> {j}')
    Energy_difference_SOC.append(Energy_SOC[j+1]-Energy_SOC[j])
    
Diff_SOC = DataFrame({
    'Transition level' : Transition_SOC,
    'Energy Differnece' : Energy_difference_SOC
})

In [206]:
print(Diff_SOC)
print(df_SOC)
df_SOC.to_csv("Transition_SOC.csv", index=False)
Diff_SOC.to_csv("rotational_SOC.csv", index=False)

  Transition level  Energy Differnece
0          1 --> 0           0.408430
1          2 --> 1           0.816861
2          3 --> 2           1.225291
3          4 --> 3           1.633721
4          5 --> 4           2.042152
5          6 --> 5           2.450582
6          7 --> 6           2.859012
7          8 --> 7           3.267443
8          9 --> 8           3.675873
   J  Energy_HF
0  0   0.000000
1  1   0.408430
2  2   1.225291
3  3   2.450582
4  4   4.084304
5  5   6.126455
6  6   8.577037
7  7  11.436050
8  8  14.703493
9  9  18.379366


**Rotation and Vibration**

In [207]:
Energy_rot_SOC = []
b = B(moment2(mo,ms,mc,roc,rsc))
d = D(moment2(mo,ms,mc,roc,rsc),roc,rsc,k)
for n in N:
    for j in J:
        Energy_rot_SOC.append(Energy(n,j,b,d))

In [208]:
df = DataFrame({
    'N': [n for n in N for _ in J],
    'J': [j for _ in N for j in J],
    'Energy_rot_SOC (1/cm)': Energy_rot_SOC
})

In [209]:
Transition_vib_rot_SOC = []
Transition_combination_SOC = []
for k in range(len(Energy_rot_SOC)):
    n1 = df.at[k, 'N']
    j1 = df.at[k,'J']
    for l in range(k,len(Energy_rot_SOC)):
        n2 = df.at[l,'N']
        j2 = df.at[l,'J']
        if abs(n1 - n2) == 1 and abs(j2 - j1) == 1:
            tr = df.at[l , 'Energy_rot_SOC (1/cm)'] - df.at[k , 'Energy_rot_SOC (1/cm)']
            Transition_vib_rot_SOC.append(tr)
            Transition_combination_SOC.append(f'{n1} --> {n2} and {j1} --> {j2}')
            
Transition_comb_SOC = DataFrame({
    'Transition' : Transition_combination_SOC,
    'Difference in Energy level (1/cm)' : Transition_vib_rot_SOC
})

In [211]:
print(D(moment2(mo,ms,mc,roc,rsc),roc,rsc,k))
print(omega(b,d))
print(df)
df.to_csv("Vibration_SOC.csv", index=False)
print(Transition_comb_SOC)
Transition_comb_SOC.to_csv("Transition_vibration_SOC.csv", index=False)

0.00031251818885936073
274.1829014785559
    N  J  Energy_rot_SOC (1/cm)
0   0  0             137.091451
1   0  1             137.499879
2   0  2             138.316725
3   0  3             139.541968
4   0  4             141.175573
5   0  5             143.217498
6   0  6             145.667689
7   0  7             148.526080
8   0  8             151.792594
9   0  9             155.467146
10  1  0             411.274352
11  1  1             411.682781
12  1  2             412.499627
13  1  3             413.724869
14  1  4             415.358474
15  1  5             417.400400
16  1  6             419.850590
17  1  7             422.708981
18  1  8             425.975496
19  1  9             429.650048
20  2  0             685.457254
21  2  1             685.865682
22  2  2             686.682528
23  2  3             687.907771
24  2  4             689.541376
25  2  5             691.583301
26  2  6             694.033492
27  2  7             696.891883
28  2  8             700.158397